# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
cities_examined = pd.read_csv("../output_data/Retrieved_City_Weather_Data.csv")
cities_examined.head()

,Unnamed: 0,City,Approx. Latitude,Approx. Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Actual Latitude,Actual Longitude,Country,Datetime (Local)
0,0,llangefni,53.191015,-4.435155,59.00,87,100,9.17,53.26,-4.31,GB,2020-06-30 21:18:46
1,1,sharlyk,53.252014,54.455958,48.02,84,5,7.76,52.92,54.75,RU,2020-07-01 01:20:52
2,2,cidreira,-43.940274,-33.017035,62.26,89,100,24.38,-30.18,-50.21,BR,2020-06-30 17:19:22
3,3,bredasdorp,-58.980568,20.953094,50.00,93,100,4.70,-34.53,20.04,ZA,2020-06-30 22:16:14
4,4,coronel oviedo,-25.457511,-56.338665,62.76,79,65,7.65,-25.42,-56.45,PY,2020-06-30 16:20:52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#enable gmaps extension for this notebook
# !pip install gmaps
# jupyter nbextension enable --py gmaps

# Configure gmaps
gmaps.configure(api_key=g_key)

#set locations (lat, long) data for the heatmap
locations = cities_examined.loc[:, ["Actual Latitude", "Actual Longitude"]]


# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=cities_examined["Humidity (%)"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
select_cities = cities_examined[(cities_examined["Cloudiness"]<20) & (cities_examined["Humidity (%)"]<30) &
                                (cities_examined["Max Temperature (F)"]>70) & (cities_examined["Max Temperature (F)"]<80)]
select_cities

,Unnamed: 0,City,Approx. Latitude,Approx. Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Actual Latitude,Actual Longitude,Country,Datetime (Local)
199,199,montrose,38.468638,-107.809077,73.40,13,1,9.17,38.42,-108.27,US,2020-06-30 14:21:37
390,390,korla,39.999715,88.308021,72.61,21,0,9.33,41.76,86.15,CN,2020-07-01 04:22:29
414,414,hami,41.797640,93.679871,72.34,25,0,4.29,42.80,93.45,CN,2020-07-01 04:22:34
442,442,khash,27.959880,62.119077,75.67,29,0,3.85,28.22,61.22,IR,2020-07-01 00:52:41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
hotel_df = select_cities.loc[:, ["City", "Actual Latitude", "Actual Longitude", "Country", "Datetime (Local)"]]

hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
#     "keyword": "hotel",
    "key": g_key
}

for i, row in hotel_df.iterrows():
    try:
        curr_lat = row["Actual Latitude"]
        curr_lng = row["Actual Longitude"]
        params["location"] = f"{curr_lat},{curr_lng}"

        base_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        name_address = requests.get(base_places_url, params)
        
        json_resp = name_address.json()
        
        hotel_name = json_resp["results"][0]["name"]
        
        hotel_df["Hotel Name"][i] = hotel_name
        
    except (KeyError, ValueError, IndexError):
        print("Hotel not found for the city of {}".format(row["City"]))

# print(json_resp["results"][0]["name"])
hotel_df

Hotel not found for the city of montrose


C:\Users\patri\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Actual Latitude,Actual Longitude,Country,Datetime (Local),Hotel Name
199,montrose,38.42,-108.27,US,2020-06-30 14:21:37,
390,korla,41.76,86.15,CN,2020-07-01 04:22:29,Gulisitan Hotel
414,hami,42.80,93.45,CN,2020-07-01 04:22:34,Laoyutang Resort
442,khash,28.22,61.22,IR,2020-07-01 00:52:41,دفتر پیشخوان دولت جمال زهی (خالد)


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
